In [1]:
import pandas as pd
import pickle
from datetime import datetime
import os
import re

In [ ]:
class CustomerDataExtractor:
    CATEGORY_MAPPING = {
        1: 'Electronics',
        2: 'Apparel',
        3: 'Books',
        4: 'Home Goods'
    }

    def __init__(self, data_path='customer_orders.pkl', vip_path='vip_customers.txt'):
        self.data_path = data_path
        self.vip_path = vip_path
        self.customer_data = None
        self.vip_ids = set()
        self.flat_data = []

    def _parse_price(self, raw_price):
        price_str = str(raw_price).replace('$', '').strip().lower()
        return 0.0 if price_str in {'', 'none', 'invalid'} else float(price_str)

    def _parse_quantity(self, raw_quantity):
        try:
            return int(raw_quantity)
        except (ValueError, TypeError):
            return 0

    def _parse_order_id(self, raw_order_id):
        if raw_order_id is None:
            return None
        match = re.search(r'\d+', str(raw_order_id))
        return int(match.group()) if match else None

    def _parse_date(self, raw_date):
        """
        Tries multiple formats to parse a date, falling back to flexible parsing.
        Handles both YYYY-MM-DD and YYYY-DD-MM layouts, with optional time.
        """
        if not raw_date:
            return pd.NaT

        fallback_formats = [
            '%Y-%m-%d %H:%M:%S',
            '%Y-%d-%m %H:%M:%S',
            '%Y-%m-%d',
            '%Y-%d-%m'
        ]

        for fmt in fallback_formats:
            try:
                return pd.to_datetime(raw_date, format=fmt)
            except (ValueError, TypeError):
                continue

        # Fallback to automatic parsing
        return pd.to_datetime(raw_date, errors='coerce')


    def load_data(self):
        with open(self.data_path, 'rb') as f:
            self.customer_data = pickle.load(f)

        with open(self.vip_path, 'r') as f:
            self.vip_ids = set(int(line.strip()) for line in f if line.strip().isdigit())

    def flatten_data(self):
        for customer in self.customer_data:
            customer_id = int(customer.get('id'))
            customer_name = str(customer.get('name'))
            registration_date = pd.to_datetime(customer.get('registration_date'))
            is_vip = customer_id in self.vip_ids

            orders = customer.get('orders', [])
            for order in orders:
                order_id = self._parse_order_id(order.get('order_id'))
                if order_id is None:
                    continue
                order_date = self._parse_date(order.get('order_date'))
                items = order.get('items', [])
                
                total_order_value = 0
                temp_items = []

                for item in items:
                    price = self._parse_price(item.get('price'))
                    quantity = self._parse_quantity(item.get('quantity'))
                    total_price = price * quantity
                    total_order_value += total_price
                    temp_items.append((item, price, quantity, total_price))

                for item, price, quantity, total_price in temp_items:
                    raw_product_id = item.get('item_id')
                    if raw_product_id is None:
                        continue
                    product_id = int(raw_product_id)
                    product_name = str(item.get('product_name'))
                    category_code = item.get('category')
                    category = self.CATEGORY_MAPPING.get(category_code, 'Misc')

                    unit_price = price
                    item_quantity = quantity
                    total_item_price = price * quantity

                    total_order_value_percentage = (
                        (total_item_price / total_order_value) * 100
                        if total_order_value > 0 else 0.0
                    )

                    self.flat_data.append({
                        'customer_id': customer_id,
                        'customer_name': customer_name,
                        'registration_date': registration_date,
                        'is_vip': is_vip,
                        'order_id': order_id,
                        'order_date': order_date,
                        'product_id': product_id,
                        'product_name': product_name,
                        'category': category,
                        'unit_price': unit_price,
                        'item_quantity': item_quantity,
                        'total_item_price': total_item_price,
                        'total_order_value_percentage': total_order_value_percentage
                    })


    def to_dataframe(self):
        df = pd.DataFrame(self.flat_data)

        df = df.astype({
            'customer_id': 'int64',
            'customer_name': 'string',
            'registration_date': 'datetime64[ns]',
            'is_vip': 'bool',
            'order_id': 'int64',
            'order_date': 'datetime64[ns]',
            'product_id': 'int64',
            'product_name': 'string',
            'category': 'string',
            'unit_price': 'float64',
            'item_quantity': 'int64',
            'total_item_price': 'float64',
            'total_order_value_percentage': 'float64'
        })

        df = df.sort_values(by=['customer_id', 'order_id', 'product_id']).reset_index(drop=True)
        return df

    def extract(self):
        self.load_data()
        self.flatten_data()
        return self.to_dataframe()

In [19]:
extractor = CustomerDataExtractor(
    data_path='customer_orders.pkl',
    vip_path='vip_customers.txt'
)
df = extractor.extract()
df.to_csv('customer_orders_flat.csv', index=False)